In [10]:
# This kernel is based on Alexander Kireev's deep learning model:
#   https://www.kaggle.com/alexanderkireev/deep-learning-support-imbalance-architect-9671
# (See notes and references below.)
# I (Andy Harless) have made the following changes:
#   1. Add 2 more (narrower) layers on top
#   2. Eliminate "day" and "wday" variables (no variation in this sample)
#   3. Change target weight from 99 to 70
#   4. Add batch normalization
#   5. Only one epoch
#   6. Eliminate weight decay
#   7. Increase batch size
#   8. Increase dropout

# version 4:  adding ipcount


# good day, my friends
# in this kernel we try to continue development of our DL models
# =================================================================================================
# we continue our work
# this kernel is attempt to configure neural network for work with imbalanced data (see ~150th row)
# =================================================================================================
# thanks for people who share his works. i hope together we can create smth interest

# https://www.kaggle.com/baomengjiao/embedding-with-neural-network
# https://www.kaggle.com/gpradk/keras-starter-nn-with-embeddings
# https://www.kaggle.com/pranav84/lightgbm-fixing-unbalanced-data-auc-0-9787
# https://www.kaggle.com/rteja1113/lightgbm-with-count-features
# https://www.kaggle.com/knowledgegrappler/a-simple-nn-solution-with-keras-0-48611-pl
# https://www.kaggle.com/isaienkov/rnn-with-keras-ridge-sgdr-0-43553
# https://www.kaggle.com/valkling/mercari-rnn-2ridge-models-with-notes-0-42755/versions#base=2202774&new=2519287

print ('Good luck')

import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '4'
import gc
from sklearn.cross_validation import train_test_split

# import libs
import numpy as np 
import itertools
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')
import warnings
warnings.filterwarnings('ignore')

from sklearn import preprocessing

path = '../input/'
dtypes = {
        'ip'            : 'uint32',
        'app'           : 'uint16',
        'device'        : 'uint16',
        'os'            : 'uint16',
        'channel'       : 'uint16',
        'is_attributed' : 'uint8',
        'click_id'      : 'uint32'
        }


print('load train....')
# we save only day 9
train_set_name = 'train_0'
train_df_full = pd.read_pickle(train_set_name)
train_df_full, train_df_del = train_test_split(train_df_full, test_size=0.8)
del train_df_del
gc.collect()

print(train_df_full.head())

Good luck
load train....
          app  device  os  channel  is_attributed  min  hour  ipcount  qty  \
8347993    12       1  32      124              0   38    23       99   99   
33676443    9       1  20      334              0   27     7      385  385   
26863117   18       1  13      134              0   11     5       27   27   
33316878    2       1  13      122              0   20     7       55   55   
4002018     2       1  47      477              0   31    18       54   54   

          ip_app_count  ip_app_os_count  
8347993            169                3  
33676443           633               27  
26863117            60                7  
33316878           107               16  
4002018             89               48  


In [12]:
# importing all the required ML packages
from sklearn.linear_model import LogisticRegression #logistic regression
from sklearn import svm #support vector Machine
from sklearn.ensemble import RandomForestClassifier #Random Forest
from sklearn.neighbors import KNeighborsClassifier #KNN
from sklearn.naive_bayes import GaussianNB #Naive bayes
from sklearn.tree import DecisionTreeClassifier #Decision Tree
from sklearn.neural_network import MLPClassifier #Neural Network
from sklearn.model_selection import train_test_split #training and testing data split
from sklearn import metrics #accuracy measure
from sklearn.metrics import confusion_matrix #for confusion matrix

# plot consufion matrix definition
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    fig=plt.gcf()
    fig.set_size_inches(10,10)

In [14]:
train,test=train_test_split(train_df_full,test_size=0.3,random_state=0,stratify=train_df_full['is_attributed'])

train_X=train.drop(['is_attributed'], axis=1)
train_Y=train[['is_attributed']]
test_X=test.drop(['is_attributed'], axis=1)
test_Y=test[['is_attributed']]
X=train_df_full.drop(['is_attributed'], axis=1)
Y=train_df_full['is_attributed']

In [16]:
print(train_X.head())
print(train_Y.head())
print(test_X.head())
print(test_Y.head())

          app  device  os  channel  min  hour  ipcount   qty  ip_app_count  \
46630400   18       1  19      107   44    11       82    82           118   
40248103   64       1  41      459   44     9       17    17            25   
33959912   12       1  13      265   33     7       69    69           144   
46389536    5       1  19      377   39    11     1456  1456            62   
2871566    15       1  19      480   22    17      110   110          1157   

          ip_app_os_count  
46630400               20  
40248103                1  
33959912               24  
46389536               18  
2871566               349  
          is_attributed
46630400              0
40248103              0
33959912              0
46389536              0
2871566               1
          app  device   os  channel  min  hour  ipcount  qty  ip_app_count  \
54409442   13       1   13      477   12    14      137  137            40   
8928946     5       1   35      377   51    23       18   18   

### SVM radial

In [ ]:
model = svm.SVC(kernel='rbf', C=1, gamma=0.1)
model.fit(train_X, train_Y) 
prediction1=model.predict(test_X)
print('Accuracy for rbf SVM is ',metrics.accuracy_score(prediction1,test_Y))

In [ ]:
model=svm.SVC(kernel='linear',C=0.1,gamma=0.1)
model.fit(train_X,train_Y)
prediction2=model.predict(test_X)
print('Accuracy for linear SVM is',metrics.accuracy_score(prediction2,test_Y))

In [ ]:
model=DecisionTreeClassifier()
model.fit(train_X,train_Y)
prediction5=model.predict(test_X)
print('The accuracy of the Decision Tree is',metrics.accuracy_score(prediction5,test_Y))

In [ ]:
model=KNeighborsClassifier() 
model.fit(train_X,train_Y)
prediction6=model.predict(test_X)
print('The accuracy of the KNN is',metrics.accuracy_score(prediction6,test_Y))

In [ ]:
model=RandomForestClassifier(n_estimators=100)
model.fit(train_X,train_Y)
prediction8=model.predict(test_X)
print('The accuracy of the Random Forests is',metrics.accuracy_score(prediction8,test_Y))

In [ ]:
y_train = train_df['is_attributed'].values

In [ ]:
print('load test...')
train_set_name = 'test'
test_df = pd.read_pickle(train_set_name)
print(test_df.head())

sub = pd.DataFrame()
sub['click_id'] = test_df['click_id'].astype('int')

def get_keras_data(dataset):
    X = {
        'app': np.array(dataset.app),
        'ch': np.array(dataset.channel),
        'dev': np.array(dataset.device),
        'os': np.array(dataset.os),
        'h': np.array(dataset.hour),
        'qty': np.array(dataset.qty),
        'ipcount': np.array(dataset.ipcount),
        'c1': np.array(dataset.ip_app_count),
        'c2': np.array(dataset.ip_app_os_count)
    }
    return X

test_df = get_keras_data(test_df)

print("predicting....")
# sub['is_attributed'] = 
del test_df; gc.collect()
print(sub.head())
print("writing....")
sub.to_csv('tddlakah1.csv', index=False, float_format='%.3f')
print("done...")